## A Simple Recommendation Engine Approach for Users of a Soccer Mobile App

This notebook will take a look at several approaches to building a content recommendation engine using User historical data.

In [50]:
#Read in the data and view it
import pandas as pd
import numpy as np
working_df = pd.read_csv('final_contcat.csv')
pd.set_option('display.max_columns', None)
working_df.head()

/Users/alexandercroft/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (17,22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Event Type,Event Timestamp,Event Arrival,Cognito ID1,Timezone,Client ID,Device,Country,Language_x,Model1,Session ID,Start Time,ID,Competition,Context,Football Awards,Football Competition,Football organisation,Football People,Football Teams,Governing Body,Omnisport Subscription,People,PRTV,Sports Technology,TournamentName,MatchID,Title,HomeTeam,AwayTeam,Source,Local,Local1,Category
0,"""ItemView""",2019-10-27 17:18:31.034,1572196711368,"""us-east-1:bef15a10-a765-436a-b843-0997407b8b8a""",Asia/Singapore,"""b90dc356-577e-4afe-8245-e6119b27e769""","""en_SG""","""SG""","""en""","""iPad""","""c1ad12d0-9B27E769-20191027-170344348""",1572195824351},video:lsVCCCG,NaN,Premier League_GoalClip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Premier League,3114156.0,Arsenal vs Crystal Palace,Arsenal,Crystal Palace,GoalServe,2019-10-27 17:18:31.034000+00:00,2019-10-28 01:18:31.034000+08:00,Video
1,"""ItemView""",2019-10-27 17:18:44.324,1572196725120,"""us-east-1:bef15a10-a765-436a-b843-0997407b8b8a""",Africa/Douala,"""9f786ade-da8c-4814-9c65-f76e472c8476""","""en_KR""","""KR""","""en""","""iPhone""","""c1ad12d0-472C8476-20191027-171517350""",1572196517351},video:GSCLIP_3314819001,NaN,Premier League_GoalClip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Premier League,3114161.0,0 - 1 H. Kane 1′,Liverpool,Tottenham Hotspur,GoalServe,2019-10-27 17:18:44.324000+00:00,2019-10-27 18:18:44.324000+01:00,Video
2,"""ItemView""",2019-10-27 17:19:18.031,1572196758233,"""us-east-1:bef15a10-a765-436a-b843-0997407b8b8a""",America/Los_Angeles,"""d62960df-16ea-4e0c-82da-cea17e4288c9""","""en_US""","""US""","""en""","""iPhone""","""c1ad12d0-7E4288C9-20191027-171845319""",1572196725319},video:GSCLIP_3314819001,NaN,Premier League_GoalClip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Premier League,3114161.0,0 - 1 H. Kane 1′,Liverpool,Tottenham Hotspur,GoalServe,2019-10-27 17:19:18.031000+00:00,2019-10-27 10:19:18.031000-07:00,Video
3,"""ItemView""",2019-10-27 17:19:39.876,1572196780118,"""us-east-1:bef15a10-a765-436a-b843-0997407b8b8a""",America/Indiana/Indianapolis,"""441423bf-f64a-420f-ad82-a8e1d7413ff5""","""en_US""","""US""","""en""","""iPhone""","""c1ad12d0-D7413FF5-20191027-171903836""",1572196743837},video:JsVCCCG,NaN,Premier League_GoalClip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Premier League,3114167.0,Norwich vs Manchester Utd,Norwich City,Manchester United,GoalServe,2019-10-27 17:19:39.876000+00:00,2019-10-27 13:19:39.876000-04:00,Video
4,"""ItemView""",2019-10-27 17:19:09.165,1572196750451,"""us-east-1:bef15a10-a765-436a-b843-0997407b8b8a""",America/New_York,"""75fdb5de-670f-4652-aa44-6cd3d757627e""","""en_US""","""US""","""en""","""iPhone""","""c1ad12d0-D757627E-20191027-170945798""",1572196185802},video:GSCLIP_3314819001,NaN,Premier League_GoalClip,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Premier League,3114161.0,0 - 1 H. Kane 1′,Liverpool,Tottenham Hotspur,GoalServe,2019-10-27 17:19:09.165000+00:00,2019-10-27 13:19:09.165000-04:00,Video


In [51]:
working_df['Context 1'] = ''
working_df['Context 2'] = ''

In [52]:
#For this analysis, we will only be looking at Premier League item views
epl_df = working_df[working_df['TournamentName'] == 'Premier League']

In [53]:
epl_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 744556 entries, 0 to 1678333
Data columns (total 36 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Event Type              744556 non-null  object 
 1   Event Timestamp         744556 non-null  object 
 2   Event Arrival           744556 non-null  int64  
 3   Cognito ID1             744556 non-null  object 
 4   Timezone                744556 non-null  object 
 5   Client ID               744556 non-null  object 
 6   Device                  744556 non-null  object 
 7   Country                 744556 non-null  object 
 8   Language_x              744556 non-null  object 
 9   Model1                  744556 non-null  object 
 10  Session ID              744556 non-null  object 
 11  Start Time              744556 non-null  object 
 12  ID                      744556 non-null  object 
 13  Competition             5147 non-null    object 
 14  Context            

In [54]:
#Keeping the columns we need
epl_df = epl_df[['Client ID','Football Teams','Football People','Football Competition','ID','Context','HomeTeam','AwayTeam','Local','Local1','Category','Context 1','Context 2']]

In [55]:
#Filtering out users that have viewed items less than 10 times
epl_df = epl_df[epl_df.groupby('Client ID')['Client ID'].transform('count') > 10]

In [56]:
epl_df.head()

,Client ID,Football Teams,Football People,Football Competition,ID,Context,HomeTeam,AwayTeam,Local,Local1,Category,Context 1,Context 2
2,"""d62960df-16ea-4e0c-82da-cea17e4288c9""",NaN,NaN,NaN,video:GSCLIP_3314819001,Premier League_GoalClip,Liverpool,Tottenham Hotspur,2019-10-27 17:19:18.031000+00:00,2019-10-27 10:19:18.031000-07:00,Video,,
3,"""441423bf-f64a-420f-ad82-a8e1d7413ff5""",NaN,NaN,NaN,video:JsVCCCG,Premier League_GoalClip,Norwich City,Manchester United,2019-10-27 17:19:39.876000+00:00,2019-10-27 13:19:39.876000-04:00,Video,,
4,"""75fdb5de-670f-4652-aa44-6cd3d757627e""",NaN,NaN,NaN,video:GSCLIP_3314819001,Premier League_GoalClip,Liverpool,Tottenham Hotspur,2019-10-27 17:19:09.165000+00:00,2019-10-27 13:19:09.165000-04:00,Video,,
5,"""599c6f23-1679-4bc2-9eb1-187db2af8113""",NaN,NaN,NaN,video:GSCLIP_3314819001,Premier League_GoalClip,Liverpool,Tottenham Hotspur,2019-10-27 17:19:23.088000+00:00,2019-10-28 01:19:23.088000+08:00,Video,,
6,"""ba5d35c0-2ba3-44ef-bdf4-c4d032d33e0a""",NaN,NaN,NaN,video:GSDIRECT_3314810,Premier League_Highlights,Newcastle United,Wolverhampton Wanderers,2019-10-27 17:19:21.933000+00:00,2019-10-27 18:19:21.933000+01:00,Video,,


In [57]:
epl_df['Context'].value_counts()

Premier League_Highlights    274317
Premier League_Featured      185290
Premier League_News          182796
Premier League_GoalClip       14817
Premier League_FullGame        3642
Premier League_nan               78
Name: Context, dtype: int64

In [58]:
epl_df['Football People'].value_counts().head(15)

Pep Guardiola             17949
Jose Mourinho             13748
Jürgen Klopp              12602
Frank Lampard              9599
Ole Gunnar Solskjaer       5196
Mauricio Pochettino        3670
Philipp Lahm               3516
Paul Pogba                 3430
Mikel Arteta Amatriain     2576
Jordan Henderson           2231
André Tavares              2030
Unai Emery                 1972
David De Gea               1843
Granit Xhaka               1627
Mesut Özil                 1564
Name: Football People, dtype: int64

In [59]:
# Video and Articles are classified differently, so we will split them into 
# separate dataframes and handle each one slightly differently.
news_df = epl_df[epl_df['ID'].str.contains('article')]
vid_df = epl_df[epl_df['ID'].str.contains('video')]

In [60]:
news_df.head()

,Client ID,Football Teams,Football People,Football Competition,ID,Context,HomeTeam,AwayTeam,Local,Local1,Category,Context 1,Context 2
11,"""81b23f95-8b70-4e74-91a2-2b46fb4288a1""",Tottenham Hotspur,Danny Rose,Premier League,article:67505,Premier League_News,NaN,NaN,2019-10-27 17:19:58.273000+00:00,2019-10-28 01:19:58.273000+08:00,News,,
21,"""6cea3c15-8e35-4806-b924-a844fe00122e""",Manchester United,Scott McTominay,Premier League,article:67509,Premier League_News,NaN,NaN,2019-10-27 17:20:56.046000+00:00,2019-10-28 01:20:56.046000+08:00,News,,
26,"""fc037c71-5241-44a2-ba7f-067627eb1779""",NaN,NaN,NaN,article:67510,Premier League_News,NaN,NaN,2019-10-27 17:21:05.751000+00:00,2019-10-27 12:21:05.751000-05:00,News,,
32,"""1130dca4-ebc3-4c67-ae3d-5e8a196989e3""",Tottenham Hotspur,Danny Rose,Premier League,article:67505,Premier League_News,NaN,NaN,2019-10-27 17:21:18.032000+00:00,2019-10-27 12:21:18.032000-05:00,News,,
39,"""977ee2ec-e8ba-41ed-b5f9-2bfcc65febc8""",Manchester United,Scott McTominay,Premier League,article:67509,Premier League_News,NaN,NaN,2019-10-27 17:21:54.829000+00:00,2019-10-27 12:21:54.829000-05:00,News,,


In [61]:
#Filter out the null values, we cannot analyze them
news_df = news_df[news_df['Football People'].notna()]
news_df = news_df[news_df['Football Competition'].notna()]
news_df = news_df[news_df['Football Teams'].notna()]

In [62]:
#Create new Context labels that will be used in our analysis
news_df['Context 1'] = news_df.apply(lambda row: row['Football People'] +'_' + row['Category'], axis=1)
news_df['Context 2'] = news_df.apply(lambda row: row['Football Teams'] +'_' + row['Category'], axis=1)


In [63]:
#Check new Dataframe
news_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140387 entries, 11 to 1678333
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Client ID             140387 non-null  object
 1   Football Teams        140387 non-null  object
 2   Football People       140387 non-null  object
 3   Football Competition  140387 non-null  object
 4   ID                    140387 non-null  object
 5   Context               140387 non-null  object
 6   HomeTeam              0 non-null       object
 7   AwayTeam              0 non-null       object
 8   Local                 140387 non-null  object
 9   Local1                140387 non-null  object
 10  Category              140387 non-null  object
 11  Context 1             140387 non-null  object
 12  Context 2             140387 non-null  object
dtypes: object(13)
memory usage: 15.0+ MB


In [64]:
vid_df.head()

,Client ID,Football Teams,Football People,Football Competition,ID,Context,HomeTeam,AwayTeam,Local,Local1,Category,Context 1,Context 2
2,"""d62960df-16ea-4e0c-82da-cea17e4288c9""",NaN,NaN,NaN,video:GSCLIP_3314819001,Premier League_GoalClip,Liverpool,Tottenham Hotspur,2019-10-27 17:19:18.031000+00:00,2019-10-27 10:19:18.031000-07:00,Video,,
3,"""441423bf-f64a-420f-ad82-a8e1d7413ff5""",NaN,NaN,NaN,video:JsVCCCG,Premier League_GoalClip,Norwich City,Manchester United,2019-10-27 17:19:39.876000+00:00,2019-10-27 13:19:39.876000-04:00,Video,,
4,"""75fdb5de-670f-4652-aa44-6cd3d757627e""",NaN,NaN,NaN,video:GSCLIP_3314819001,Premier League_GoalClip,Liverpool,Tottenham Hotspur,2019-10-27 17:19:09.165000+00:00,2019-10-27 13:19:09.165000-04:00,Video,,
5,"""599c6f23-1679-4bc2-9eb1-187db2af8113""",NaN,NaN,NaN,video:GSCLIP_3314819001,Premier League_GoalClip,Liverpool,Tottenham Hotspur,2019-10-27 17:19:23.088000+00:00,2019-10-28 01:19:23.088000+08:00,Video,,
6,"""ba5d35c0-2ba3-44ef-bdf4-c4d032d33e0a""",NaN,NaN,NaN,video:GSDIRECT_3314810,Premier League_Highlights,Newcastle United,Wolverhampton Wanderers,2019-10-27 17:19:21.933000+00:00,2019-10-27 18:19:21.933000+01:00,Video,,


In [65]:
#Filter the nulls and add context labels
vid_df = vid_df[vid_df['HomeTeam'].notna()]
vid_df = vid_df[vid_df['AwayTeam'].notna()]
vid_df = vid_df[vid_df['Category'].notna()]

vid_df['Context 1'] = vid_df.apply(lambda row: row['HomeTeam'] +'_' + row['Context'], axis=1)
vid_df['Context 2'] = vid_df.apply(lambda row: row['AwayTeam'] +'_' + row['Context'], axis=1)


In [66]:
vid_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 448544 entries, 2 to 1678297
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Client ID             448544 non-null  object
 1   Football Teams        0 non-null       object
 2   Football People       0 non-null       object
 3   Football Competition  0 non-null       object
 4   ID                    448544 non-null  object
 5   Context               448544 non-null  object
 6   HomeTeam              448544 non-null  object
 7   AwayTeam              448544 non-null  object
 8   Local                 448544 non-null  object
 9   Local1                448544 non-null  object
 10  Category              448544 non-null  object
 11  Context 1             448544 non-null  object
 12  Context 2             448544 non-null  object
dtypes: object(13)
memory usage: 47.9+ MB


In [67]:
#Add the dataframes together 
new_df = vid_df.append(news_df)

In [68]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 588931 entries, 2 to 1678333
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Client ID             588931 non-null  object
 1   Football Teams        140387 non-null  object
 2   Football People       140387 non-null  object
 3   Football Competition  140387 non-null  object
 4   ID                    588931 non-null  object
 5   Context               588931 non-null  object
 6   HomeTeam              448544 non-null  object
 7   AwayTeam              448544 non-null  object
 8   Local                 588931 non-null  object
 9   Local1                588931 non-null  object
 10  Category              588931 non-null  object
 11  Context 1             588931 non-null  object
 12  Context 2             588931 non-null  object
dtypes: object(13)
memory usage: 62.9+ MB


In [69]:
new_df.head()

,Client ID,Football Teams,Football People,Football Competition,ID,Context,HomeTeam,AwayTeam,Local,Local1,Category,Context 1,Context 2
2,"""d62960df-16ea-4e0c-82da-cea17e4288c9""",NaN,NaN,NaN,video:GSCLIP_3314819001,Premier League_GoalClip,Liverpool,Tottenham Hotspur,2019-10-27 17:19:18.031000+00:00,2019-10-27 10:19:18.031000-07:00,Video,Liverpool_Premier League_GoalClip,Tottenham Hotspur_Premier League_GoalClip
3,"""441423bf-f64a-420f-ad82-a8e1d7413ff5""",NaN,NaN,NaN,video:JsVCCCG,Premier League_GoalClip,Norwich City,Manchester United,2019-10-27 17:19:39.876000+00:00,2019-10-27 13:19:39.876000-04:00,Video,Norwich City_Premier League_GoalClip,Manchester United_Premier League_GoalClip
4,"""75fdb5de-670f-4652-aa44-6cd3d757627e""",NaN,NaN,NaN,video:GSCLIP_3314819001,Premier League_GoalClip,Liverpool,Tottenham Hotspur,2019-10-27 17:19:09.165000+00:00,2019-10-27 13:19:09.165000-04:00,Video,Liverpool_Premier League_GoalClip,Tottenham Hotspur_Premier League_GoalClip
5,"""599c6f23-1679-4bc2-9eb1-187db2af8113""",NaN,NaN,NaN,video:GSCLIP_3314819001,Premier League_GoalClip,Liverpool,Tottenham Hotspur,2019-10-27 17:19:23.088000+00:00,2019-10-28 01:19:23.088000+08:00,Video,Liverpool_Premier League_GoalClip,Tottenham Hotspur_Premier League_GoalClip
6,"""ba5d35c0-2ba3-44ef-bdf4-c4d032d33e0a""",NaN,NaN,NaN,video:GSDIRECT_3314810,Premier League_Highlights,Newcastle United,Wolverhampton Wanderers,2019-10-27 17:19:21.933000+00:00,2019-10-27 18:19:21.933000+01:00,Video,Newcastle United_Premier League_Highlights,Wolverhampton Wanderers_Premier League_Highlights


## Approach 1: Recommend labels using each user's individual behavior

In [70]:
#Viewing each users top viewed context labels
client_df = pd.DataFrame(new_df.groupby('Client ID')['Context 1'].value_counts())


In [71]:
client_df.head(45)

Context 1
Client ID                              Context 1                                                  
"001d1700-92bd-4bed-8ddf-887be687c2eb" Aston Villa_Premier League_Highlights                     3
                                       Crystal Palace_Premier League_Highlights                  3
                                       Southampton_Premier League_Highlights                     3
                                       AFC Bournemouth_Premier League_Highlights                 2
                                       Newcastle United_Premier League_Highlights                2
                                       Newcastle_Premier League_Highlights                       2
                                       Tottenham Hotspur_Premier League_Highlights               2
"00201187-6334-4d80-b1ac-b4ac77049d95" Brighton & Hove Albion_Premier League_Highlights          3
                                       Liverpool_Premier League_Highlights                       3
                                       Manchester City_Premier League_Highlights                 3
                                       Newcastle_Premier League_Highlights                       3
                                       Hugo Lloris_News                                          2
                                       Jürgen Klopp_News                                         2
                                       Laurent Koscielny_News                                    2
                                       Manchester City_Premier League_Featured                   2
                                       Norwich City_Premier League_Highlights                    2
                                       Pep Guardiola_News                                        2
                                       AFC Bournemouth_Premier League_Highlights                 1
                                       Arsenal_Premier League_Highlights                         1
                                       Aston Villa_Premier League_Highlights                     1
                                       Ben Foster_News                                           1
                                       Everton_Premier League_Highlights                         1
                                       Granit Xhaka_News                                         1
                                       Harry Kane_News                                           1
                                       Ilkay Gündogan_News                                       1
                                       Jose Mourinho_News                                        1
                                       Newcastle United_Premier League_Highlights                1
                                       Neymar_News                                               1
                                       Pierre Aubameyang_News                                    1
                                       Roy Keane_News                                            1
                                       Sokratis Papastathopoulos_News                            1
                                       Southampton_Premier League_Highlights                     1
                                       Tottenham Hotspur_Premier League_Highlights               1
"0023ff32-3a57-4f74-a235-ff958b2e06ea" Liverpool_Premier League_Featured                        15
                                       Liverpool_Premier League_Highlights                       3
                                       Chelsea_Premier League_Featured                           2
                                       Everton_Premier League_Featured                           2
                                       Leicester City_Premier League_Featured                    2
                                       Leicester City_Premier League_Highlights                  2
"00244d0e-5e8f-46d6-9dd4-8b37b

In [72]:
#Cleaning it up a bit!
client_1_df = client_df.rename(columns={'Context 1': 'Count'})
client_1_df = client_1_df.reset_index()
client_1_df['Client ID'] = client_1_df['Client ID'].str.replace('"','')
client_1_df.head()

,Client ID,Context 1,Count
0,001d1700-92bd-4bed-8ddf-887be687c2eb,Aston Villa_Premier League_Highlights,3
1,001d1700-92bd-4bed-8ddf-887be687c2eb,Crystal Palace_Premier League_Highlights,3
2,001d1700-92bd-4bed-8ddf-887be687c2eb,Southampton_Premier League_Highlights,3
3,001d1700-92bd-4bed-8ddf-887be687c2eb,AFC Bournemouth_Premier League_Highlights,2
4,001d1700-92bd-4bed-8ddf-887be687c2eb,Newcastle United_Premier League_Highlights,2


In [73]:
#Function that will take a Client's ID and output their top 3 viewed Labels.
def top2(clientID,df) :
    df = df[df['Client ID'] == clientID]
    df = df.reset_index()
    max_1 = df['Context 1'][0]
    try :
        max_2 = df['Context 1'][1]
    except:
        max_2 = False
    try :
        max_3 = df['Context 1'][2]
    except:
        max_3 = False
    return [clientID,max_1,max_2,max_3]

print(top2('003e8e18-b963-4a48-8790-2b0d0ddcdea7',client_1_df))

['003e8e18-b963-4a48-8790-2b0d0ddcdea7', 'Liverpool_Premier League_Highlights', 'Liverpool_Premier League_Featured', 'Norwich City_Premier League_Highlights']


In [74]:
client_1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 283378 entries, 0 to 283377
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Client ID  283378 non-null  object
 1   Context 1  283378 non-null  object
 2   Count      283378 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 6.5+ MB


In [75]:
#List of users
clients = client_1_df['Client ID'].unique()
#Create a dataframe of users and their top 3 labels
cols = ['Client ID','1st','2nd','3rd']
final_df = pd.DataFrame(columns = cols)
final_df.head()

,Client ID,1st,2nd,3rd


In [76]:
#Using our top3 function, create the Dataframe
for client in clients:
    lst = top2(client,client_1_df)
    df_length = len(final_df)
    final_df.loc[df_length] = lst



final_df.head(20)

,Client ID,1st,2nd,3rd
0,001d1700-92bd-4bed-8ddf-887be687c2eb,Aston Villa_Premier League_Highlights,Crystal Palace_Premier League_Highlights,Southampton_Premier League_Highlights
1,00201187-6334-4d80-b1ac-b4ac77049d95,Brighton & Hove Albion_Premier League_Highlights,Liverpool_Premier League_Highlights,Manchester City_Premier League_Highlights
2,0023ff32-3a57-4f74-a235-ff958b2e06ea,Liverpool_Premier League_Featured,Liverpool_Premier League_Highlights,Chelsea_Premier League_Featured
3,00244d0e-5e8f-46d6-9dd4-8b37bbd420e0,Leicester City_Premier League_Featured,Liverpool_Premier League_Featured,Manchester City_Premier League_Featured
4,0030367e-4155-4698-a9f7-15507579f0ed,Frank Lampard_News,Jose Mourinho_News,Arsenal_Premier League_Highlights
5,003845f9-cbad-43a4-be3e-2cf5da016143,AFC Bournemouth_Premier League_Highlights,Sheffield United_Premier League_Highlights,Aston Villa_Premier League_Highlights
6,003e8e18-b963-4a48-8790-2b0d0ddcdea7,Liverpool_Premier League_Highlights,Liverpool_Premier League_Featured,Norwich City_Premier League_Highlights
7,003ed19f-50bf-4d71-9323-eaf0de932839,Chelsea_Premier League_Featured,Leicester City_Premier League_Featured,AFC Bournemouth_Premier League_Featured
8,00478d7e-18c3-4844-bdfa-b67680cad771,Pep Guardiola_News,Crystal Palace_Premier League_Highlights,Chelsea_Premier League_Highlights
9,00500352-4825-4358-a33f-f9d2b8539648,Leicester City_Premier League_Highlights,Manchester City_Premier League_Highlights,Aston Villa_Premier League_Highlights


This Dataframe shows each unique user in the dataset and their top viewed context label. This could be used to recommend articles/videos to each user if the item is uploaded with those corresponding labels. It's worth nothing that this could easily be expanded to top 5, top 10 context labels viewed.

In [77]:
client_1_df['Client ID'].ne('Count').idxmax()


0

## Approach Number 2: KNN Model to recommend labels using Cosine Similarity

In [78]:
#Create a sparse matrix to be used in a KNN model
pivot_df = pd.crosstab(vid_df['Client ID'],vid_df['Context 1'])


In [79]:
pivot_df.head(45)

Context 1,AFC Bournemouth_Premier League_Featured,AFC Bournemouth_Premier League_FullGame,AFC Bournemouth_Premier League_GoalClip,AFC Bournemouth_Premier League_Highlights,Akhmat Grozny_Premier League_Highlights,Arsenal Tula_Premier League_GoalClip,Arsenal Tula_Premier League_Highlights,Arsenal_Premier League_Featured,Arsenal_Premier League_FullGame,Arsenal_Premier League_GoalClip,Arsenal_Premier League_Highlights,Aston Villa_Premier League_Featured,Aston Villa_Premier League_FullGame,Aston Villa_Premier League_GoalClip,Aston Villa_Premier League_Highlights,Black Leopards_Premier League_Highlights,Bohemians_Premier League_GoalClip,Bohemians_Premier League_nan,Brighton & Hove Albion_Premier League_Featured,Brighton & Hove Albion_Premier League_FullGame,Brighton & Hove Albion_Premier League_GoalClip,Brighton & Hove Albion_Premier League_Highlights,Burnley_Premier League_Featured,Burnley_Premier League_FullGame,Burnley_Premier League_GoalClip,Burnley_Premier League_Highlights,Burnley_Premier League_nan,CSKA Moscow_Premier League_GoalClip,CSKA Moscow_Premier League_Highlights,Cardiff City_Premier League_Featured,Cardiff City_Premier League_FullGame,Celtic_Premier League_GoalClip,Celtic_Premier League_Highlights,Chelsea_Premier League_Featured,Chelsea_Premier League_FullGame,Chelsea_Premier League_GoalClip,Chelsea_Premier League_Highlights,Cork City_Premier League_GoalClip,Crystal Palace_Premier League_Featured,Crystal Palace_Premier League_FullGame,Crystal Palace_Premier League_GoalClip,Crystal Palace_Premier League_Highlights,Dinamo Moscow_Premier League_GoalClip,Dinamo Moscow_Premier League_Highlights,Everton_Premier League_Featured,Everton_Premier League_FullGame,Everton_Premier League_GoalClip,Everton_Premier League_Highlights,Everton_Premier League_nan,Fulham_Premier League_Featured,Hearts_Premier League_Highlights,Huddersfield Town_Premier League_Featured,Krasnodar_Premier League_GoalClip,Krasnodar_Premier League_Highlights,Krylya Sovetov_Premier League_GoalClip,Leicester City_Premier League_Featured,Leicester City_Premier League_FullGame,Leicester City_Premier League_GoalClip,Leicester City_Premier League_Highlights,Liverpool_Premier League_Featured,Liverpool_Premier League_FullGame,Liverpool_Premier League_GoalClip,Liverpool_Premier League_Highlights,Lokomotiv Moscow_Premier League_Highlights,Manchester City_Premier League_Featured,Manchester City_Premier League_FullGame,Manchester City_Premier League_GoalClip,Manchester City_Premier League_Highlights,Manchester United_Premier League_Featured,Manchester United_Premier League_FullGame,Manchester United_Premier League_GoalClip,Manchester United_Premier League_Highlights,Newcastle United_Premier League_Featured,Newcastle United_Premier League_FullGame,Newcastle United_Premier League_GoalClip,Newcastle United_Premier League_Highlights,Newcastle_Premier League_GoalClip,Newcastle_Premier League_Highlights,Newcastle_Premier League_nan,Norwich City_Premier League_Featured,Norwich City_Premier League_FullGame,Norwich City_Premier League_GoalClip,Norwich City_Premier League_Highlights,Orenburg_Premier League_Highlights,Rostov_Premier League_GoalClip,Rostov_Premier League_Highlights,Rubin Kazan_Premier League_GoalClip,Rubin Kazan_Premier League_Highlights,Shamrock Rovers_Premier League_GoalClip,Sheffield United_Premier League_Featured,Sheffield United_Premier League_FullGame,Sheffield United_Premier League_GoalClip,Sheffield United_Premier League_Highlights,Sochi_Premier League_GoalClip,Sochi_Premier League_Highlights,Southampton_Premier League_Featured,Southampton_Premier League_FullGame,Southampton_Premier League_GoalClip,Southampton_Premier League_Highlights,Spartak Moscow_Premier League_GoalClip,Spartak Moscow_Premier League_Highlights,St Patrick's Athletic_Premier League_GoalClip,Tambov_Premier League_Highlights,Tottenham Hotspur_Premier League_Featured,Tottenham Hotspur_Premier League_FullGame,Tottenham Hotspur_Premier League_GoalClip,Tottenham Hotspur_Premier League_Highlights

In [80]:
from scipy.sparse import csr_matrix
matrix_df = csr_matrix(pivot_df.values)

In [81]:
#Define nearest neighbors cosine model
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
model_knn = NearestNeighbors(metric='cosine',algorithm='brute',n_neighbors=20)
model_knn.fit(matrix_df)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [82]:
def recommender(context):
    idx = process.extractOne(context,vid_df['Context 1'])
    print(idx)
    
recommender('AFC Bournemouth')

('AFC Bournemouth_Premier League_Featured', 90, 877)


In [83]:
#Define a function that will recommend the labels using KNN
def recc(context,data,model,n_recc):
    model.fit(data)
    idx = process.extractOne(context,vid_df['Context 1'])[2]
    print('Context Selected: ',vid_df['Context 1'][idx], 'Index ',idx)
    print('Searching....')
    distances, indices=model.kneighbors(data[idx],n_neighbors = n_recc)
    print(distances,indices)
    
    
    
    
recc('Chelsea',matrix_df,model_knn,20)


Context Selected:  Chelsea_Premier League_Featured Index  437
Searching....
[[0.         0.14341385 0.17246461 0.18010841 0.18350342 0.19935923
  0.20943058 0.21045797 0.21665055 0.22405971 0.23490794 0.23939127
  0.24407105 0.24407105 0.24622164 0.24660099 0.24803921 0.24807938
  0.25       0.25164457]] [[  437  6615 10134  3819  4607 12735  9902 13369 10113  8090  2763 11456
   1056  5057  1802 10742  7705 13747  6033  4286]]


In [84]:
#View the context labels that are recommended
che_recc = [437,6615,10134,3819,4607,12735,9902,13369,10113,8090,2763,11456,1056,5057,1802,10742,7705,13747,6033,4286]

for i in che_recc:
    print(vid_df['Context 1'][i-1:i])
    
#print(vid_df['Context 1'][437])

2028    Newcastle United_Premier League_Featured
Name: Context 1, dtype: object
28490    Huddersfield Town_Premier League_Featured
Name: Context 1, dtype: object
43092    Burnley_Premier League_Highlights
Name: Context 1, dtype: object
15989    Newcastle United_Premier League_Highlights
Name: Context 1, dtype: object
18383    Sheffield United_Premier League_Highlights
Name: Context 1, dtype: object
55607    Liverpool_Premier League_Featured
Name: Context 1, dtype: object
42307    Liverpool_Premier League_Highlights
Name: Context 1, dtype: object
59020    Manchester City_Premier League_Highlights
Name: Context 1, dtype: object
43005    Sheffield United_Premier League_Highlights
Name: Context 1, dtype: object
34830    Manchester United_Premier League_Highlights
Name: Context 1, dtype: object
12285    Sheffield United_Premier League_Featured
Name: Context 1, dtype: object
49520    Arsenal_Premier League_Featured
Name: Context 1, dtype: object
4550    Southampton_Premier League_Highlights


The KNN approach doesn't seem to work as well, with the cosine similarity scores between labels quite low. This approach needs some additional work.

## Approach Number 3: Recommend labels based on correlation of label vectors

In [85]:
#Correlation score approach
def corr_item(matrix,context):
    context_df = matrix[context]
    similar = matrix.corrwith(context_df)
    return similar

In [86]:
#Check the function works
Che_feat = corr_item(pivot_df,'Chelsea_Premier League_Featured')
print(Che_feat)

Context 1
AFC Bournemouth_Premier League_Featured              0.500649
AFC Bournemouth_Premier League_FullGame              0.006960
AFC Bournemouth_Premier League_GoalClip             -0.010152
AFC Bournemouth_Premier League_Highlights            0.134067
Akhmat Grozny_Premier League_Highlights              0.015735
                                                       ...   
Wolverhampton Wanderers_Premier League_FullGame      0.012424
Wolverhampton Wanderers_Premier League_GoalClip     -0.008901
Wolverhampton Wanderers_Premier League_Highlights    0.198430
Zenit_Premier League_GoalClip                        0.033936
Zenit_Premier League_Highlights                      0.013405
Length: 126, dtype: float64


In [87]:
corr_df = pd.DataFrame(Che_feat,columns =['Correlation'])

In [88]:
#Check correlation label and corresponding correlation score
corr_df = corr_df.sort_values(by='Correlation',ascending=False)
corr_df.head(30)

,Correlation
Context 1,
Chelsea_Premier League_Featured,1.000000
Leicester City_Premier League_Featured,0.647907
Arsenal_Premier League_Featured,0.620918
Tottenham Hotspur_Premier League_Featured,0.612692
Manchester City_Premier League_Featured,0.586259
Manchester United_Premier League_Featured,0.539596
Everton_Premier League_Featured,0.536489
Burnley_Premier League_Featured,0.531517
Newcastle United_Premier League_Featured,0.522308


This approach looks at each label as a vector and assigns a correlation score based on the similarity of one vector to another. As expected, the label correlates perfectly with itself and that's why it gets a 1.0 score. The rest of the vectors scores are shown above. This appproach may be useful in recommending items after the user views an item with that label.

## Approach Number 4: Recommend labels by favorite team

In [89]:
#Categorize users into fan bases based on their top viewed label
che_users = []
liv_users = []
manc_users = []
manu_users = []
for user in final_df['Client ID']:
    if 'Chelsea' in top2(user,client_1_df)[1]:
        che_users.append(user)
    elif 'Liverpool' in top2(user,client_1_df)[1]:
        liv_users.append(user)
    elif 'Manchester City' in top2(user,client_1_df)[1]:
        manc_users.append(user)
    elif 'Manchester United' in top2(user,client_1_df)[1]:
        manu_users.append(user)

In [90]:
#View a list of Chelsea fans
print(che_users)

['003ed19f-50bf-4d71-9323-eaf0de932839', '0074001e-fdef-4d0e-9aa0-eea43107917b', '00c11cc3-c7ee-4613-897d-693199efc6f2', '00d10f78-aae7-43fe-80b5-03c69e49cf4c', '01316c60-44cd-4f6a-bb9a-d3ac9fd62ee0', '0137bb46-9b59-4c6a-855a-3141d92583dd', '018a8b4e-3253-40f7-bf4e-512991e5a44c', '019094f6-daf4-4197-8729-bc602a9dcfcd', '01b41c8f-c434-4303-8360-ee842f13faf8', '01bd3a5f-0fab-44b3-b09b-fa1222c73c72', '0226ceca-2b04-4521-8f61-12c8c9180d04', '024de66e-f228-403b-9ad3-61414b4ca721', '026c1ce2-a51b-4057-bfe7-ddc42cf06b78', '02e7eb5b-6dfa-469f-9a86-b00cb03534f4', '03079627-4511-4aa7-a2b1-885753738094', '04f3b51c-fa6e-4c20-aad0-9db97f73eb38', '05e2dde6-b997-4143-a128-df2f1022b4d7', '05ea6c52-6d02-492a-ac53-4ff5c36245fd', '06370f5a-99c9-40d9-8a1d-a6ff8dd95b60', '0651fbc6-2810-450e-91e4-d9edd634a050', '06e57bb9-9d74-44cb-8bb8-0a5552026c98', '06f59395-5687-42f2-882d-0349e810b400', '0723fb36-9156-4877-96eb-60f53b2b053c', '0732c1b5-e5e3-451c-ac3b-207c2ca711b2', '079bb71f-9f02-4f01-b3e3-6209a7b450a4',

In [91]:
#Check to make sure a user in the Chelsea dataframe does have a Chelsea label as their top label
top2('09d230e6-7416-45c1-865e-5507c1e5990d',client_1_df)

['09d230e6-7416-45c1-865e-5507c1e5990d',
 'Chelsea_Premier League_Featured',
 'Manchester City_Premier League_Featured',
 'Manchester United_Premier League_Featured']

In [92]:
#Create the dataframes
che_df = final_df[final_df['Client ID'].isin(che_users)]
liv_df = final_df[final_df['Client ID'].isin(liv_users)]
manc_df = final_df[final_df['Client ID'].isin(manc_users)]
manu_df = final_df[final_df['Client ID'].isin(manu_users)]


In [93]:
#Double check that the users have Chelsea in their 1st label column
che_df.head()

,Client ID,1st,2nd,3rd
7,003ed19f-50bf-4d71-9323-eaf0de932839,Chelsea_Premier League_Featured,Leicester City_Premier League_Featured,AFC Bournemouth_Premier League_Featured
16,0074001e-fdef-4d0e-9aa0-eea43107917b,Chelsea_Premier League_Highlights,Everton_Premier League_Highlights,Manchester City_Premier League_Highlights
33,00c11cc3-c7ee-4613-897d-693199efc6f2,Chelsea_Premier League_Featured,Chelsea_Premier League_Highlights,Newcastle United_Premier League_Featured
38,00d10f78-aae7-43fe-80b5-03c69e49cf4c,Chelsea_Premier League_Highlights,Manchester City_Premier League_Highlights,Leicester City_Premier League_Highlights
53,01316c60-44cd-4f6a-bb9a-d3ac9fd62ee0,Chelsea_Premier League_Featured,Leicester City_Premier League_Featured,Liverpool_Premier League_Featured


In [94]:
#View each fan bases 2nd most viewed label, after their own team
che_df['2nd'].value_counts()

Leicester City_Premier League_Highlights     59
Liverpool_Premier League_Highlights          58
Liverpool_Premier League_Featured            56
Manchester City_Premier League_Featured      41
Arsenal_Premier League_Highlights            38
                                             ..
Southampton_Premier League_FullGame           1
Cardiff City_Premier League_Featured          1
Jürgen Klopp_News                             1
Jesse Marsch_News                             1
Tottenham Hotspur_Premier League_FullGame     1
Name: 2nd, Length: 63, dtype: int64

In [95]:
manc_df['2nd'].value_counts().head()

Manchester United_Premier League_Featured      129
Liverpool_Premier League_Featured              126
Arsenal_Premier League_Featured                117
Manchester United_Premier League_Highlights    103
Arsenal_Premier League_Highlights              101
Name: 2nd, dtype: int64

In [96]:
manu_df['2nd'].value_counts()

Manchester City_Premier League_Featured      94
Arsenal_Premier League_Highlights            88
Liverpool_Premier League_Highlights          83
Liverpool_Premier League_Featured            78
Norwich City_Premier League_Highlights       76
                                             ..
Adam Lallana_News                             1
Tottenham Hotspur_Premier League_FullGame     1
West Ham United_Premier League_GoalClip       1
Granit Xhaka_News                             1
Frank Lampard_News                            1
Name: 2nd, Length: 90, dtype: int64

In [97]:
liv_df['2nd'].value_counts()

Leicester City_Premier League_Highlights     335
Manchester City_Premier League_Featured      303
Liverpool_Premier League_Highlights          294
Manchester City_Premier League_Highlights    257
Arsenal_Premier League_Highlights            221
                                            ... 
Sokratis Papastathopoulos_News                 1
Roy Keane_News                                 1
Aston Villa_Premier League_FullGame            1
Sheffield United_Premier League_GoalClip       1
False                                          1
Name: 2nd, Length: 103, dtype: int64

In [98]:
#Organize into a Dataframe and output the top 20 recommended labels by each top 4 team
top_4 = pd.DataFrame()
top_4['Liv Context'] = np.asarray(liv_df['2nd'].value_counts().head(20).index)
top_4['Liv Count'] = np.asarray(liv_df['2nd'].value_counts().head(20))
                                
top_4['Manc Context'] = np.asarray(manc_df['2nd'].value_counts().head(20).index)
top_4['Manc Count'] = np.asarray(manc_df['2nd'].value_counts().head(20))
                                
top_4['Manu Context'] = np.asarray(manu_df['2nd'].value_counts().head(20).index)
top_4['Manu Count'] = np.asarray(manu_df['2nd'].value_counts().head(20))
                                
                                
top_4['Che Context'] = np.asarray(che_df['2nd'].value_counts().head(20).index)
top_4['Che Count'] = np.asarray(che_df['2nd'].value_counts().head(20))
top_4.head(20)

,Liv Context,Liv Count,Manc Context,Manc Count,Manu Context,Manu Count,Che Context,Che Count
0,Leicester City_Premier League_Highlights,335,Manchester United_Premier League_Featured,129,Manchester City_Premier League_Featured,94,Leicester City_Premier League_Highlights,59
1,Manchester City_Premier League_Featured,303,Liverpool_Premier League_Featured,126,Arsenal_Premier League_Highlights,88,Liverpool_Premier League_Highlights,58
2,Liverpool_Premier League_Highlights,294,Arsenal_Premier League_Featured,117,Liverpool_Premier League_Highlights,83,Liverpool_Premier League_Featured,56
3,Manchester City_Premier League_Highlights,257,Manchester United_Premier League_Highlights,103,Liverpool_Premier League_Featured,78,Manchester City_Premier League_Featured,41
4,Arsenal_Premier League_Highlights,221,Arsenal_Premier League_Highlights,101,Norwich City_Premier League_Highlights,76,Arsenal_Premier League_Highlights,38
5,Manchester United_Premier League_Featured,200,Liverpool_Premier League_Highlights,99,Arsenal_Premier League_Featured,72,Manchester City_Premier League_Highlights,34
6,AFC Bournemouth_Premier League_Highlights,199,Tottenham Hotspur_Premier League_Highlights,59,Sheffield United_Premier League_Highlights,58,Manchester United_Premier League_Featured,30
7,Arsenal_Premier League_Featured,173,Tottenham Hotspur_Premier League_Featured,54,Tottenham Hotspur_Premier League_Highlights,54,Arsenal_Premier League_Featured,28
8,Manchester United_Premier League_Highlights,170,Aston Villa_Premier League_Highlights,47,Manchester United_Premier League_Highlights,53,Manchester United_Premier League_Highlights,27
9,Aston Villa_Premier League_Highlights,144,Southampton_Premier League_Highlights,42,Leicester City_Premier League_Highlights,45,Everton_Premier League_Highlights,24


This approach looks at each fan bases top viewed labels other than their team. This approach looks to work well, as each fan base is viewing teams that are in close competition with each other. It's interesting to see that the Manchester fans are viewing each others items the most, as they are close rivals. Chelsea fans are viewing Leceister the most, the team they were chasing for 3rd place the entire season. And of course, everyone is keeping their eyes on Liverpool, who held a commanding lead of the title race all season.